<h1 style="text-align: center;">Confluence Knowledge Assistant</h1>
<p style="text-align: center;"><em>Powered by Google ADK &amp; Atlassian MCP</em></p>

**This agent lets you chat naturally with Confluence-based product documentation or other information stored in Confluence.** It reads a Markdown index file (a simple list of page titles, IDs, and links) once and stores it in the tool context state, then uses it to look up the most relevant Confluence pages and, in this example, generate accurate answers limited strictly to the Marmind Knowledge Base.

### Prerequisites

To run the **Confluence Knowledge Assistant** with an MCP connection, you need:

1. Python 3.13 with a virtual environment (`venv`).
2. JupyterLab (installed inside the venv).
3. Node.js - includes `npx`, which is used to launch the MCP client (`mcp-remote`) on the fly without a global install.
4. Access to the Atlassian MCP endpoint: `https://mcp.atlassian.com/v1/sse`

In [ ]:
!pip install google-adk mcp

In [1]:
import os
import json
import contextlib
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_toolset import StdioConnectionParams, StdioServerParameters

In [12]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
if GEMINI_API_KEY:
    print("API keys loaded!")
else:
    print("Google API key missing!")

API keys loaded!


**`read_md_file`** is a helper tool that loads a Markdown index file (`marmind_kb_table_short.md`) into the agent’s session state the first time it’s called. It caches the file content in `tool_context.state` under `md_content` and sets a flag (`md_read`) so subsequent calls skip re-reading the file and just return the cached content. This ensures the agent always has quick access to the index without repeatedly hitting the filesystem.

> Keep the table as small as possible (e.g. shorten summaries, strip formatting) so you don’t waste tokens.
> If the table grows too large, replace it with an embedding / vector lookup step instead of dumping the whole file into the LLM each time.

In [13]:
async def read_md_file(tool_context: ToolContext) -> str:
    """
    Reads the mapping table markdown file once and caches it.
    Always returns the content so the model can use it.
    """
    # If already cached, return directly
    if tool_context.state.get("md_read", False):
        return tool_context.state["md_content"]

    file_path = "marmind_kb_table_short.md"
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        tool_context.state["md_content"] = content
        tool_context.state["md_read"] = True

        return content
    except FileNotFoundError:
        return "❌ markdown file not found."
    except Exception as e:
        return f"❌ Error reading file: {e}"

In [14]:
# Agent with MCPToolset that auto-discovers Atlassian tools
agent = Agent(
    name="product_doc_assistant",
    model="gemini-2.5-flash",
    description="Confluence documentation assistant with auto-discovered Atlassian MCP tools for marmind-knowledgebase.atlassian.net",
    instruction="""
        You are a specialized AI assistant designed to help users with documentation for the **Marmind** product. Your knowledge is strictly limited to the **'Marmind Knowledge Base' Confluence space** on `marmind-knowledgebase.atlassian.net`. You are **not permitted** to access or search any other Confluence spaces, resources, or external information.
    
        **Your Goal**:
        - Provide accurate, concise, and helpful answers based *only* on the provided Confluence documentation.
    
        **Your Workflow**:
        Always follow these steps precisely!
        1.  **Analyze the user's question** to determine their specific intent.
        2.  **Use {md_content|""} to review the documentation mapping table and identify the one or two most relevant pages based on the user's query. Only call `read_md_file` if {md_content|""} is empty.
        3.  Using the **Page ID** from the mapping table, **call `getConfluencePage`** to retrieve the content of the selected page(s).
        4.  **Formulate your answer** using *only* the information retrieved from the Confluence page content.
        5.  **Always include the direct link** to the knowledgebase for each page following this format: "For more details, refer to: [Page Title](Link)"
    
        **Crucial Rules**:
        - **Source Restriction:** You must use **only** the content from the retrieved Confluence pages.
        - **Scope Restriction:** You are **forbidden** from accessing, searching, or using information from any Confluence space other than **'Marmind Knowledge Base'**.
        - **Tooling:** Your access is limited to `read_md_file` and `getConfluencePage`. You are **not permitted** to use `confluence_search` or any other search-related tools.
        - **Handling Out-of-Scope Requests:** If a user's question is not related to the InsightHub documentation, or if they attempt to access other Confluence spaces or perform actions outside of your defined workflow, you must politely decline the request. Explain that your capabilities are strictly limited to providing information from the **'Marmind Knowledge Base'** Confluence space.
    
        Be helpful. Be accurate. Be effective.
        """,
    tools=[ # Combining MCP server tools with local agent tools is unreliable, since ambiguous questions may cause the agent to call the wrong tool (when the Agent is provided with many local and MCP tools). It’s better to use two connected agents.
        read_md_file,
        
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=[
                        '-y',  # Auto-confirm npx install
                        'mcp-remote',
                        'https://mcp.atlassian.com/v1/sse'
                    ],
                ),
                timeout=60 
            ),
            tool_filter=['getConfluenceSpaces', 'getConfluencePage'] 
        )      
    ],
)


# Setup session and runner  
session_service = InMemorySessionService()
runner = Runner(agent=agent, app_name="product_doc_assistant", session_service=session_service)

# Global session setup
USER_ID = "user"
SESSION_ID = "product_doc_conversation"
session_initialized = False


async def chat():
    """Start interactive conversation with the agent"""
    global session_initialized
    
    if not session_initialized:
        await session_service.create_session(
            app_name="product_doc_assistant",
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        session_initialized = True
        print("🤖 Hello! I'm here to assist with your documentation needs. Please ask me a question about the product, or type 'quit', 'exit', 'stop' to end our chat.")
    
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("👋 Goodbye!")
            break
            
        if not user_input:
            continue
            
        message = types.Content(role='user', parts=[types.Part(text=user_input)])
        print("Agent: ", end="", flush=True)
        
        with open(os.devnull, 'w') as f, contextlib.redirect_stderr(f):
            async for event in runner.run_async(
                user_id=USER_ID,
                session_id=SESSION_ID,
                new_message=message
            ):
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if part.text:
                            print(part.text, end="", flush=True)
        
        print("\n")

# [DEBUG] Use this to print the full execution output
# async def chat():
#     """Start interactive conversation with the agent"""
#     global session_initialized
    
#     if not session_initialized:
#         await session_service.create_session(
#             app_name="product_doc_assistant",
#             user_id=USER_ID,
#             session_id=SESSION_ID
#         )
#         session_initialized = True
#         print("🤖 Hello! I'm here to assist with your documentation needs. Please ask me a question about the product, or type 'quit' to end our chat.")
    
#     while True:
#         user_input = input("You: ").strip()
        
#         if user_input.lower() in ['quit', 'exit', 'stop']:
#             print("👋 Goodbye!")
#             break
            
#         if not user_input:
#             continue
            
#         message = types.Content(role='user', parts=[types.Part(text=user_input)])
#         print("Agent: ", end="", flush=True)
        
#         events = []
#         async for event in runner.run_async(
#             user_id=USER_ID,
#             session_id=SESSION_ID,
#             new_message=message
#         ):
#             events.append(event)
#             print("\n────────── RAW EVENT ──────────")
#             print(json.dumps(event.model_dump(), indent=2, default=str))
#             print(
#                 f"[{type(event).__name__:<25}] "
#                 f"author={event.author:<15} "
#                 f"final={event.is_final_response()}"
#             )
            
#             # Print content parts (LLM output)
#             if event.content and event.content.parts:
#                 for part in event.content.parts:
#                     if part.text:
#                         print("💬 text →", part.text.strip())
            
#             # Tool calls
#             for call in event.get_function_calls():
#                 print(f"🔧 tool‐call → {call.name}({json.dumps(call.args, indent=2)})")
            
#             # Tool results
#             for resp in event.get_function_responses():
#                 print(f"[TOOL] tool‐result → {resp.name} → {json.dumps(resp.response.model_dump() if hasattr(resp.response, 'model_dump') else str(resp.response), indent=2, default=str)}")
            
#             # State delta
#             if event.actions and event.actions.state_delta:
#                 print("[STATE] state Δ →", json.dumps(event.actions.state_delta, indent=2))
#             if event.actions and event.actions.artifact_delta:
#                 print("[ARTIFACT] artifact Δ →", json.dumps(event.actions.artifact_delta, indent=2))
        
#         # Final output
#         final_text = (
#             events[-1].content.parts[0].text
#             if events and events[-1].content and events[-1].content.parts else ""
#         )
#         print("\n===== ✅ FINAL ANSWER =====")
#         print(final_text or "(no text)")
#         print("\n")

In [ ]:
await chat()

🤖 Hello! I'm here to assist with your documentation needs. Please ask me a question about the product, or type 'quit', 'exit', 'stop' to end our chat.


You:  tell me what the Marmind product is all about


Agent: MARMIND is a Marketing Resource Management (MRM) platform designed to help marketing teams plan, execute, and optimize their activities. It provides a central place to manage marketing plans, budgets, and performance data, reducing the need for multiple disconnected tools.

With Marmind, teams can:
*   **Plan** marketing activities using an interactive calendar.
*   **Collaborate** across departments with shared tasks and workflows.
*   **Manage content and digital assets** in a central repository.
*   **Track budgets and spending** in real time.
*   **Monitor performance** through automated reporting and AI-driven insights.

Key features include Marketing Planning, Cross-Team Collaboration, Content & Digital Assets management, Budget & Spend Management, Performance & Control, Workflow Automation, and User Roles and Access Control.

For more details, refer to: [Marmind Overview and Basics](https://knowledgebase.marmind.com/kb/marmind-overview)



In [6]:
# Remove all MCP auth tokens. This forces re-authentication next time
# !rm -rf ~/.mcp-auth

In [7]:
# Remove all persistent MCP/Atlassian data
# !rm -rf ~/.mcp* ~/.atlassian* ~/.config/mcp* ~/.config/atlassian* ~/.cache/mcp* ~/.cache/atlassian*